# AB Test and Bidding Methods Conversion Comparison

## Business Problem
### Facebook recently introduced a new bidding type, 'average bidding', as an alternative to the existing bidding type called 'maximum bidding'.
### One of our customers, bombabomba.com, decided to test this new feature and would like to do an A/B test to see if average bidding converts more than maximum bidding.
### The A/B test has been going on for 1 month and bombabomba.com is now waiting for you to analyze the results of this A/B test. The ultimate success criterion for Bombabomba.com is Purchase. Therefore, the focus should be on the Purchase metric for statistical testing.

## Dataset Story

### In this data set, which includes the website information of a company, there is information such as the number of advertisements that users see and click, as well as earnings information from here. There are two separate data sets, the control and test groups. These datasets are in separate sheets of the ab_testing.xlsx excel. Maximum Bidding for the control group, Average for the test group Bidding has been applied.

* Impression: Ad views
* Click: Number of clicks on the displayed ad
* Purchase: The number of products purchased after the ads clicked
* Earning: Earnings after purchased products

In [1]:
# Task 1: Preparing and Analyzing Data

In [2]:
# Step 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

In [39]:
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import statsmodels.stats.api as sms
import scipy.stats as st
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore") 

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [11]:
df_control = pd.read_excel("ab_testing.xlsx", sheet_name="Control Group")
df_test = pd.read_excel("ab_testing.xlsx", sheet_name="Test Group")

In [12]:
df_control.head()

,Impression,Click,Purchase,Earning,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,82529.45927,6090.07732,665.21125,2311.27714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,98050.45193,3382.86179,315.08489,1742.80686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82696.02355,4167.96575,458.08374,1797.82745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109914.40040,4910.88224,487.09077,1696.22918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,108457.76263,5987.65581,441.03405,1543.72018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_test.head()

,Impression,Click,Purchase,Earning,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,120103.50380,3216.54796,702.16035,1939.61124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,134775.94336,3635.08242,834.05429,2929.40582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,107806.62079,3057.14356,422.93426,2526.24488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,116445.27553,4650.47391,429.03353,2281.42857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,145082.51684,5201.38772,749.86044,2781.69752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_control.shape

(40, 14)

In [15]:
df_test.shape

(40, 14)

In [18]:
# Let's look at the dataset

In [16]:
df_control.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,101711.44907,20302.15786,45475.94296,85726.69035,99790.70108,115212.81654,147539.33633
Click,40.00000,5100.65737,1329.98550,2189.75316,4124.30413,5001.22060,5923.80360,7959.12507
Purchase,40.00000,550.89406,134.10820,267.02894,470.09553,531.20631,637.95709,801.79502
Earning,40.00000,1908.56830,302.91778,1253.98952,1685.84720,1975.16052,2119.80278,2497.29522
Unnamed: 4,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 7,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 9,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00000,120512.41176,18807.44871,79033.83492,112691.97077,119291.30077,132050.57893,158605.92048
Click,40.00000,3967.54976,923.09507,1836.62986,3376.81902,3931.35980,4660.49791,6019.69508
Purchase,40.00000,582.10610,161.15251,311.62952,444.62683,551.35573,699.86236,889.91046
Earning,40.00000,2514.89073,282.73085,1939.61124,2280.53743,2544.66611,2761.54540,3171.48971
Unnamed: 4,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 5,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 6,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 7,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 8,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 9,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Editing the names of control and test group variables

In [20]:
df_control.columns = [i + "_control" for i in df_control.columns]
df_control.head()

,Impression_control,Click_control,Purchase_control,Earning_control,Unnamed: 4_control,Unnamed: 5_control,Unnamed: 6_control,Unnamed: 7_control,Unnamed: 8_control,Unnamed: 9_control,Unnamed: 10_control,Unnamed: 11_control,Unnamed: 12_control,Unnamed: 13_control
0,82529.45927,6090.07732,665.21125,2311.27714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,98050.45193,3382.86179,315.08489,1742.80686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82696.02355,4167.96575,458.08374,1797.82745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109914.40040,4910.88224,487.09077,1696.22918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,108457.76263,5987.65581,441.03405,1543.72018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_test.columns = [i + "_test" for i in df_test.columns]
df_test.head()

,Impression_test,Click_test,Purchase_test,Earning_test,Unnamed: 4_test,Unnamed: 5_test,Unnamed: 6_test,Unnamed: 7_test,Unnamed: 8_test,Unnamed: 9_test,Unnamed: 10_test,Unnamed: 11_test,Unnamed: 12_test,Unnamed: 13_test
0,120103.50380,3216.54796,702.16035,1939.61124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,134775.94336,3635.08242,834.05429,2929.40582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,107806.62079,3057.14356,422.93426,2526.24488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,116445.27553,4650.47391,429.03353,2281.42857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,145082.51684,5201.38772,749.86044,2781.69752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Step 2: Analyze control and test group data.
# In the Impression breakdown, by taking the sums of the clicks and the average of the purchases into account, a table has emerged according to the impressions.

In [26]:
df_control.groupby("Impression_control").agg({"Click_control": "sum",
                              "Purchase_control": "mean"})

,Click_control,Purchase_control
Impression_control,,
45475.94296,7370.21438,400.18883
77773.63390,4462.20659,519.66966
79498.24866,6653.84552,470.50137
80254.33164,3075.31120,530.67988
81872.98572,7959.12507,729.16155
82067.89994,6609.18343,267.02894
82529.45927,6090.07732,665.21125
82696.02355,4167.96575,458.08374
83676.60243,7153.97419,487.82877


In [28]:
df_test.groupby("Impression_test").agg({"Click_test": "sum",
                              "Purchase_test": "mean"})

,Click_test,Purchase_test
Impression_test,,
79033.83492,4495.42818,425.35910
79234.91193,6002.21358,382.04712
83356.58756,6019.69508,570.43213
96331.36543,3860.62836,889.91046
97507.36685,4119.21862,670.52139
102257.45409,4800.06832,521.31073
104971.22276,3563.51505,699.28728
106116.43664,3279.47297,491.61453
107806.62079,3057.14356,422.93426


In [30]:
# Step 3: After the analysis process, combine the control and test group data using the concat method.
# In this section, control and test data combined and It's becoming much more easier to analyze.

In [33]:
df_last = pd.concat([df_control, df_test], axis=1)
df_last.head()

,Impression_control,Click_control,Purchase_control,Earning_control,Unnamed: 4_control,Unnamed: 5_control,Unnamed: 6_control,Unnamed: 7_control,Unnamed: 8_control,Unnamed: 9_control,Unnamed: 10_control,Unnamed: 11_control,Unnamed: 12_control,Unnamed: 13_control,Impression_test,Click_test,Purchase_test,Earning_test,Unnamed: 4_test,Unnamed: 5_test,Unnamed: 6_test,Unnamed: 7_test,Unnamed: 8_test,Unnamed: 9_test,Unnamed: 10_test,Unnamed: 11_test,Unnamed: 12_test,Unnamed: 13_test
0,82529.45927,6090.07732,665.21125,2311.27714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120103.50380,3216.54796,702.16035,1939.61124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,98050.45193,3382.86179,315.08489,1742.80686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134775.94336,3635.08242,834.05429,2929.40582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82696.02355,4167.96575,458.08374,1797.82745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107806.62079,3057.14356,422.93426,2526.24488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109914.40040,4910.88224,487.09077,1696.22918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116445.27553,4650.47391,429.03353,2281.42857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,108457.76263,5987.65581,441.03405,1543.72018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145082.51684,5201.38772,749.86044,2781.69752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# For analyzing Confidence Intervals, using sms.DescrStatsW will be used.
print(sms.DescrStatsW(df_control["Purchase_control"]).tconfint_mean())
print(sms.DescrStatsW(df_test["Purchase_test"]).tconfint_mean())

(508.0041754264924, 593.7839421139709)
(530.5670226990063, 633.645170597929)


In [34]:
# Task 2: Defining the Hypothesis of A/B Testing

In [35]:
# Adım 1: Defining the hypothesis.
# H0 : M1 = M2
# H1 : M1!= M2 

In [43]:
# Assumption of Normality
# Variance Homogeneity

# H0: There is no statistically significant difference between Average bidding and Maximum bidding in Purchase averages.
# H1 There is a statistically significant difference between Average bidding and Maximum bidding in Purchase averages

In [44]:
# Step 2: Analyze the purchase averages for the control and test group.

In [46]:
df_last[["Purchase_control", "Purchase_test"]].mean()

Purchase_control   550.89406
Purchase_test      582.10610
dtype: float64

In [50]:
# When outcomes examines, It's seen that the new promotion type which is "average bidding" haf an affect. Otherwise, we cannot say it's enough to present it as an outcome. Surely we should keep going through our hypothesis.

In [48]:
# Task 3: Performing Hypothesis Testing

Step 1: Perform hypothesis checks before hypothesis testing.
These are Assumption of Normality and Homogeneity of Variance. Test separately whether the control and test groups comply with the normality assumption, over the Purchase variable.
Normality Assumption:
H0: Normal distribution assumption is provided.
H1: The assumption of normal distribution is not provided.
p < 0.05 H0 REJECT , p > 0.05 H0 CANNOT REJECT
Is the assumption of normality according to the test result provided for the control and test groups? Interpret the p-values obtained.
Variance Homogeneity:
H0: Variances are homogeneous.
H1: Variances are not homogeneous.
p < 0.05 H0 REJECT , p > 0.05 H0 CANNOT REJECT
Test whether the homogeneity of variance is provided for the control and test groups over the Purchase variable.
Is the assumption of normality provided according to the test result? Interpret the p-values obtained.

In [56]:
#Normality Assumption (Shapiro-Wilks Test)

# For Control
test_stat, pvalue = shapiro(df_last["Purchase_control"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [57]:
# For Test
test_stat, pvalue = shapiro(df_last["Purchase_test"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


If we look at the results of the Assumption of Normality, the p-values for both control and test groups are greater than 0.05. In this case, the result is that the H0 hypothesis cannot be rejected because the p-value is greater than 0.05. Since H0 could not be rejected, the assumption of normality was provided.

In [58]:
 # Step 2: Select the appropriate test according to the Normality Assumption and Variance Homogeneity results.

In [59]:
# Variance Homogeneity (Levene Test)
# H0: Variances are homogeneous.
# H1: Variances are not homogeneous.

In [60]:
test_stat, pvalue = levene(df_last["Purchase_control"],
                           df_last["Purchase_test"])
print("Test stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test stat = 2.6393, p-value = 0.1083


Two samples t (Parametric test) The p-value value obtained as a result of Levene's test is greater than 0.05 as observed. The result in this case is that H0 is undeniable. The H0 hypothesis reveals that the variances are homogeneous. According to the result, we could not reject H0 and variance homogeneity was achieved.

In [61]:
# Step 3: Considering the p_value obtained as a result of the test, interpret whether there is a statistically significant difference between the purchasing averages of the control and test groups.

Since we cannot reject the assumption hypotheses, the process will continue with the Parametric test.

In [62]:
test_stat, pvalue = ttest_ind(df_last["Purchase_control"],
                              df_last["Purchase_test"],
                              equal_var=True)
print("Test stat = %.4f, p-value = %.4f" % (test_stat, pvalue))
# The reason of "equal_var = True" is the variance homogeneity.

Test stat = -0.9416, p-value = 0.3493


The p-value resulting from the parametric test is 0.3493. If the P-value is higher than 0.05, it shows that the H0 hypothesis is undeniable. When the values were analyzed within the scope of the project, it was seen that the homogeneity of the variances and the H0 hypotheses were undeniable. The fact that the H0 hypothesis cannot be rejected in the parametric test, which is the last test conducted in this context, proves the statement that "There is no statistically significant difference between the Purchase averages of average bidding and Max. bidding". In the first analysis made within the scope of this project, we said that there is a difference between the newly developed model and the previous model, but that we should examine whether the result is statistically by chance. This result shows that there is no statistically significant difference between Purchase averages of Average bidding and Maximum bidding.